# Esse NoteBook serve para avaliar inicialmente quais atributos são relevantes e necessários para o atingimento da seleção.

Inicialmente serão testado os seguintes atributos:


1.   Classe: identifica se o débito é previdenciário ou fazendário; 
2.   Sit_Cadastral: indica a situação cadastral do contribuinte;
3.   Diferenciada: indica se a empresa é diferenciada, grande porte;
4.   Especial: indica se a empresa é especial;
5.   Simei: indica se a empresa é Simples ou MEI;
6.   Simples: indica se a empresa é do simples;
7.   Porte: indica o porte da empresa;
8.   Grupo_Nat_Juridica: Grupo da Natureza Jurídica;
9.   Nat_Juridica: Natureza Jurídica;
10.  Setor_Economico: Setor econômico;
11.  Cnae1: Seção da Tabela CNAE;
12.  Cnae2: Divisão da Tabela CNAE
13.  Cnae3: Grupo da Tabela CNAE
14.  Cnae4: Classe da Tabela CNAE



Importações iniciais das bibliotecas que serão utilizadas

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder

Carregar base de dados inicial

In [ ]:
dados_202012 = pd.read_excel("/content/sample_data/TCC_Extracao_inicial_todas_colunas.xlsx", sheet_name=0) 

Tratamento de dados


    Faz os seguintes tratamentos:
    Remove os registros com os valores:
        setor_economico <> 'N/A'
        cnae1 <> 'INVÁLIDO'
        cnae1 <> 'NÃO INFORMADO'
    Trata os campos de valor para transformar em números    


In [ ]:
print(dados_202012.shape)
print(dados_202012.keys())

# dropna remove os registros faltantes
dados_202012.dropna(axis=0, subset=['sit_cadastral'], inplace = True) 
dados_202012.dropna(axis=0, subset=['setor_economico'], inplace = True)
dados_202012.dropna(axis=0, subset=['cnae3'], inplace = True)

#Remove registros que tem CNAE Inválido ou Não Informado
dados_202012['sit_cadastral'] = dados_202012['sit_cadastral'].astype('str')
dados_202012 = dados_202012[~dados_202012['sit_cadastral'].str.contains('NULA')]

#Remove registros que tem CNAE Inválido ou Não Informado
dados_202012['cnae3'] = dados_202012['cnae3'].astype('str')
dados_202012 = dados_202012[~dados_202012['cnae3'].str.contains('INVÁLIDO')]
dados_202012 = dados_202012[~dados_202012['cnae3'].str.contains('NÃO INFORMADO')]

#Coloca zero quando não tem valor, para não dá erro no calculo da diferença. Quando está nulo é por que não tem mais débito, ou seja, está zerado
dados_202012[['valorm1', 'valorm2', 'valorm3']] = dados_202012[['valorm1', 'valorm2', 'valorm3']].fillna(value = 0)

print(dados_202012.shape)
print(dados_202012.keys())

(30239, 18)
Index(['classe', 'sit_cadastral', 'diferenciada', 'especial', 'simei',
       'simples', 'porte', 'grupo_nat_juridica', 'nat_juridica',
       'setor_economico', 'cnae1', 'cnae2', 'cnae3', 'cnae4', 'valor',
       'valorm1', 'valorm2', 'valorm3'],
      dtype='object')
(29953, 18)
Index(['classe', 'sit_cadastral', 'diferenciada', 'especial', 'simei',
       'simples', 'porte', 'grupo_nat_juridica', 'nat_juridica',
       'setor_economico', 'cnae1', 'cnae2', 'cnae3', 'cnae4', 'valor',
       'valorm1', 'valorm2', 'valorm3'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Criando campos de comparação

In [ ]:
# Cria os campos de comparação com o mês base, calculando a diferença na evolução de cada mês
dados_202012["Dif1"] = dados_202012.valorm1 - dados_202012.valor
dados_202012["Dif2"] = dados_202012.valorm2 - dados_202012.valor
dados_202012["Dif3"] = dados_202012.valorm3 - dados_202012.valor

print(dados_202012.shape)
print(dados_202012.keys())

(29953, 21)
Index(['classe', 'sit_cadastral', 'diferenciada', 'especial', 'simei',
       'simples', 'porte', 'grupo_nat_juridica', 'nat_juridica',
       'setor_economico', 'cnae1', 'cnae2', 'cnae3', 'cnae4', 'valor',
       'valorm1', 'valorm2', 'valorm3', 'Dif1', 'Dif2', 'Dif3'],
      dtype='object')


Exporta arquivo Formatado.=

In [ ]:
dados_202012.to_excel("/content/sample_data/TCC_Extracao_inicial_todas_colunas_FORMATADO.xlsx", index=False)

# Depois do formatamento inicial dos dados, começa a preparação para a análise

Método de encode para definir a variação dos débito

In [ ]:
def encode_units(x):
  if x < -100:
    return "Diminuiu"
  elif x > 100:
    return "Aumentou"
  else:
      return "Manteve"

Esses são os campos do DataSet que serão analisados

  0 - classe	
  1 - sit_cadastral	
  2 - diferenciada	
  3 - especial	
  4 - simei	
  4 - simples	
  6 - porte	
  7 - grupo_nat_juridica	
  8 - nat_juridica	
  9 - setor_economico	
  10 - cnae1	
  11 - cnae2	
  12 - cnae3	
  13 - cnae4  
  14 - valor	
  15 - valorm1	
  16 - valorm2	
  17 - valorm3	
  18 - Dif1  
  19 - Dif2	
  20 - Dif3

In [ ]:
pd_debitos = pd.read_excel("/content/sample_data/TCC_Extracao_inicial_todas_colunas_FORMATADO.xlsx", sheet_name=0) 
print("\nDimensões: {0}".format(pd_debitos.shape))
print("\nCampos: {0}".format(pd_debitos.keys()))
print("\nQuantidade de Registros antes: {0}".format(pd_debitos["classe"].count()))

#Tranforma os valores de diferença em -100 se diminuiu e 100 se aumentou o débito
pd_debitos["Dif3"] = pd_debitos["Dif3"].apply(encode_units)

#Faz o LabelEncode para todos os atributos
var_mod =['classe','sit_cadastral','diferenciada','especial','simei','simples', 'porte', 'grupo_nat_juridica', 'nat_juridica', 'setor_economico', 'cnae1', 'cnae2', 'cnae3', 'cnae4'] 
le = LabelEncoder() 
for i in var_mod: 
  pd_debitos[i] = le.fit_transform(pd_debitos[i]) 


#define a coluna de resultado
y_train = le.fit_transform(pd_debitos.iloc[:,20:21])
print("\nOs Labels 0, 1 e 2 são : {0}".format(le.inverse_transform([0,1,2])))


#Define as colunas que serão removidas
coluna_removidas = ['especial', 'grupo_nat_juridica', 'setor_economico','cnae1', 'cnae2', 'cnae3'] #['diferenciada','especial', 'grupo_nat_juridica', 'setor_economico','cnae1', 'cnae2', 'cnae3']
pd_debitos.drop(coluna_removidas, axis = 1, inplace=True)
print("\nTodos os Campos: {0}".format(pd_debitos.keys()))

#define o treinamento apenas com os atributos restantes
X_train = pd_debitos.iloc[:,0:14-len(coluna_removidas)]
print("\nCampos restantes: {0}".format(pd_debitos.iloc[:,0:14-len(coluna_removidas)].keys()))

#Treina e verifica a acurácia
debitos_tree = DecisionTreeClassifier(random_state=0)
debitos_tree = debitos_tree.fit(X_train, y_train) #comando que irá treinar é o FIT
print("Acurácia (base de treinamento):", debitos_tree.score(X_train, y_train))

Train_predict = debitos_tree.predict(X_train)
print(classification_report(y_train, Train_predict, target_names=["Aumentou", "Diminuiu", "Manteve"]))


Dimensões: (29953, 21)

Campos: Index(['classe', 'sit_cadastral', 'diferenciada', 'especial', 'simei',
       'simples', 'porte', 'grupo_nat_juridica', 'nat_juridica',
       'setor_economico', 'cnae1', 'cnae2', 'cnae3', 'cnae4', 'valor',
       'valorm1', 'valorm2', 'valorm3', 'Dif1', 'Dif2', 'Dif3'],
      dtype='object')

Quantidade de Registros antes: 29953

Os Labels 0, 1 e 2 são : ['Aumentou' 'Diminuiu' 'Manteve']

Todos os Campos: Index(['classe', 'sit_cadastral', 'diferenciada', 'simei', 'simples', 'porte',
       'nat_juridica', 'cnae4', 'valor', 'valorm1', 'valorm2', 'valorm3',
       'Dif1', 'Dif2', 'Dif3'],
      dtype='object')

Campos restantes: Index(['classe', 'sit_cadastral', 'diferenciada', 'simei', 'simples', 'porte',
       'nat_juridica', 'cnae4'],
      dtype='object')
Acurácia (base de treinamento): 0.9984308750375589
              precision    recall  f1-score   support

    Aumentou       0.99      1.00      1.00      7013
    Diminuiu       1.00      1.00    

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
